In [2]:
#Loding Libraries and Dependencies

import numpy as np
import tensorflow as tf
from scipy.io import loadmat
from keras.models import Model
from keras.models import model_from_json, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
import h5py
import pandas as pd


In [3]:
#Loading the pre-trained weights from .mat model
file2='/home/rishi/Projects/Matroid/vgg networks/vgg_face_matconvnet/vgg_face_matconvnet/data/vgg_face.mat'
data=loadmat(file2)

In [5]:
#HandWritten Model-For extracting weights
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), name= 'conv1_1'))
model.add(Activation('relu', name='relu1_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), name= 'conv1_2'))
model.add(Activation('relu', name='relu1_2'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool1'))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), name= 'conv2_1'))
model.add(Activation('relu', name='relu2_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), name= 'conv2_2'))
model.add(Activation('relu', name='relu2_2'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool2'))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), name= 'conv3_1'))
model.add(Activation('relu', name='relu3_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), name= 'conv3_2'))
model.add(Activation('relu', name='relu3_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), name= 'conv3_3'))
model.add(Activation('relu', name='relu3_3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool3'))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv4_1'))
model.add(Activation('relu', name='relu4_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv4_2'))
model.add(Activation('relu', name='relu4_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv4_3'))
model.add(Activation('relu', name='relu4_3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool4'))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv5_1'))
model.add(Activation('relu', name='relu5_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv5_2'))
model.add(Activation('relu', name='relu5_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv5_3'))
model.add(Activation('relu', name='relu5_3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool5'))
 
model.add(Convolution2D(4096, (7, 7), name= 'fc6'))
model.add(Activation('relu', name='relu6'))
model.add(Dropout(0.5, name='dropout6'))
model.add(Convolution2D(4096, (1, 1), name= 'fc7'))
model.add(Activation('relu', name='relu7'))
model.add(Dropout(0.5, name='dropout7'))
model.add(Convolution2D(2622, (1, 1), name= 'fc8'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Activation('softmax', name= 'softmax'))

In [7]:
#net consists of layers, weights, biases
net=data['net'][0][0]
ref_model_layers=net['layers']

In [66]:
#Ensuring that only Convolutional and fc layers are being considered
#for extracting weights. Pooling Layers and Relu will give errors.
for i in range(ref_model_layers.shape[1]):
    ref_model_layer=ref_model_layers[0][i][0][0]['name'][0]
    
    try:
        weights=ref_model_layers[0][i][0][0]['weights'][0][0]
        print(ref_model_layer, ":", weights.shape)
    except:
        print("",end="")

conv1_1 : (3, 3, 3, 64)
conv1_2 : (3, 3, 64, 64)
conv2_1 : (3, 3, 64, 128)
conv2_2 : (3, 3, 128, 128)
conv3_1 : (3, 3, 128, 256)
conv3_2 : (3, 3, 256, 256)
conv3_3 : (3, 3, 256, 256)
conv4_1 : (3, 3, 256, 512)
conv4_2 : (3, 3, 512, 512)
conv4_3 : (3, 3, 512, 512)
conv5_1 : (3, 3, 512, 512)
conv5_2 : (3, 3, 512, 512)
conv5_3 : (3, 3, 512, 512)
fc6 : (7, 7, 512, 4096)
fc7 : (1, 1, 4096, 4096)
fc8 : (1, 1, 4096, 2622)


In [77]:
#Checking if MATLAB files and our vars contain the same layers or not
#If they do, extract weights and biases to store in Keras format.

base_model_layer_names = [layer.name for layer in model.layers]
num_of_ref_model_layers = ref_model_layers.shape[1]
for i in range(num_of_ref_model_layers):
    ref_model_layer=ref_model_layers[0][i][0][0]['name'][0]
    if ref_model_layer in base_model_layer_names:
        #we just need to set convolution and fully connected weights
        if ref_model_layer.find("conv") == 0 or ref_model_layer.find("fc") == 0:
            print(i,". ",ref_model_layer)
            base_model_index = base_model_layer_names.index(ref_model_layer)

            weights=ref_model_layers[0][i][0][0]['weights'][0][0]
            bias = ref_model_layers[0][i][0][0]['weights'][0][1]

            model.layers[base_model_index].set_weights([weights, bias[:,0]])

0 .  conv1_1
2 .  conv1_2
5 .  conv2_1
7 .  conv2_2
10 .  conv3_1
12 .  conv3_2
14 .  conv3_3
17 .  conv4_1
19 .  conv4_2
21 .  conv4_3
24 .  conv5_1
26 .  conv5_2
28 .  conv5_3
31 .  fc6
34 .  fc7
37 .  fc8


In [83]:
#Saving the weights and Model for Keras
model.save_weights("vgg_pretrained_weights.h5")
model_config=model.to_json()
open("model_structure.json", "w").write(model_config)
model.save("model.hdf5")